In [1]:
import sklearn
import matplotlib.pyplot as plt
import seaborn as sb
from keras.models import load_model
import os
import pandas as pd
import PIL
import numpy as np
from PIL import Image
from matplotlib import image
import cv2 as cv
import keras
import tensorflow as tf


2023-05-05 08:37:33.120472: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 08:37:35.622451: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-05 08:37:43.236181: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-05 08:37:43.236351: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

# determining Presence or Absence of tumors

In [2]:
#Loading datafile using keras utils

Directory_Path = '/home/richard/Desktop/BtainTumorDetectionSystem/BigDataset'
Brain_State_Data=tf.keras.utils.image_dataset_from_directory(Directory_Path)

Found 13416 files belonging to 2 classes.


2023-05-05 08:37:54.113952: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-05 08:37:54.113995: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kali): /proc/driver/nvidia/version does not exist
2023-05-05 08:37:54.196003: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Preprocessing

In [3]:
#Scalling


Data = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function = tf.keras.applications.vgg16.preprocess_input)\
.flow_from_directory(directory = Directory_Path,target_size = (256,256), classes = ['Brain Tumor','Healthy'],
                     batch_size =32 )

scaler = tf.keras.layers.experimental.preprocessing.Rescaling(1./256)

Found 13503 images belonging to 2 classes.


In [4]:
#Data = scaler()

In [5]:
#Data pipeling techniques
#To display the datasets

data_iterator = Brain_State_Data.as_numpy_iterator()

Image_batch = data_iterator.next()
len(Image_batch)

2

In [6]:
len(Image_batch[0])

32

In [7]:
#Data.size

In [8]:
#Data.classes

In [9]:
Image_batch[0].shape

(32, 256, 256, 3)

In [10]:
Image_batch[1].shape

(32,)

In [11]:
len(Brain_State_Data)

420

In [12]:
#Image processing

#Image Splitting into Train,Validation,Test

Train_Data = int(len(Brain_State_Data)*.6)
Validation_Data = int(len(Brain_State_Data)*.3)
Test_Data = int(len(Brain_State_Data)*.1)


In [13]:
Train = Brain_State_Data.take(Train_Data)
Validation = Brain_State_Data.skip(Train_Data).take(Validation_Data)
Test = Brain_State_Data.skip(Train_Data + Validation_Data).take(Test_Data)

In [14]:
len(Train)

252

In [15]:
len(Validation)

126

In [16]:
#Creating a Sequential model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense,Flatten

Model = Sequential()

Model.add(Conv2D(32,(3,3),1,activation = 'relu',padding = 'same',input_shape = (256,256,3)))
Model.add(MaxPooling2D())

Model.add(Conv2D(16,(3,3),1,activation = 'relu'))
Model.add(MaxPooling2D())

Model.add(Conv2D(32,(3,3),1,activation = 'relu'))
Model.add(MaxPooling2D())

Model.add(Flatten())

Model.add(Dense(256,activation = 'relu'))
Model.add(Dense(1,activation = 'sigmoid'))
Model.compile('adam',loss = tf.losses.BinaryCrossentropy(),metrics = ['accuracy'])

In [17]:
Model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 126, 126, 16)      4624      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 63, 63, 16)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 61, 61, 32)        4640      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 32)       0

In [ ]:
#Creating a log directory
Log_dir = '/home/richard/Desktop/BtainTumorDetectionSystem/Project_Logs'

#creating Callbacks
Tensorboard_Callback = tf.keras.callbacks.TensorBoard(log_dir = Log_dir)
Model_Hist = Model.fit(Train, epochs = 60,validation_data = Validation,callbacks = [Tensorboard_Callback])

Epoch 1/60
252/252 [==============================] - 149s 584ms/step - loss: 2.9771 - accuracy: 0.8421 - val_loss: 0.1829 - val_accuracy: 0.9318
Epoch 2/60
252/252 [==============================] - 77s 305ms/step - loss: 0.1114 - accuracy: 0.9613 - val_loss: 0.1301 - val_accuracy: 0.9514
Epoch 3/60
252/252 [==============================] - 77s 305ms/step - loss: 0.0437 - accuracy: 0.9849 - val_loss: 0.0751 - val_accuracy: 0.9747
Epoch 4/60
252/252 [==============================] - 77s 305ms/step - loss: 0.0202 - accuracy: 0.9942 - val_loss: 0.0558 - val_accuracy: 0.9844
Epoch 5/60
252/252 [==============================] - 77s 306ms/step - loss: 0.0114 - accuracy: 0.9971 - val_loss: 0.0869 - val_accuracy: 0.9789
Epoch 6/60
252/252 [==============================] - 78s 307ms/step - loss: 0.0123 - accuracy: 0.9963 - val_loss: 0.0861 - val_accuracy: 0.9804
Epoch 7/60
252/252 [==============================] - 77s 307ms/step - loss: 0.0223 - accuracy: 0.9923 - val_loss: 0.1251 - val_a

2023-05-05 09:05:36.934637: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 1 of 256
2023-05-05 09:05:39.380752: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 82 of 256
2023-05-05 09:05:43.448043: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


252/252 [==============================] - 411s 2s/step - loss: 1.2144e-04 - accuracy: 1.0000 - val_loss: 0.0919 - val_accuracy: 0.9844
Epoch 18/60
252/252 [==============================] - 92s 338ms/step - loss: 6.3169e-05 - accuracy: 1.0000 - val_loss: 0.0929 - val_accuracy: 0.9849
Epoch 19/60
252/252 [==============================] - 80s 316ms/step - loss: 4.6491e-05 - accuracy: 1.0000 - val_loss: 0.1025 - val_accuracy: 0.9841
Epoch 20/60
252/252 [==============================] - 81s 321ms/step - loss: 3.5703e-05 - accuracy: 1.0000 - val_loss: 0.0996 - val_accuracy: 0.9846
Epoch 21/60
252/252 [==============================] - 80s 317ms/step - loss: 2.7672e-05 - accuracy: 1.0000 - val_loss: 0.1016 - val_accuracy: 0.9846
Epoch 22/60
252/252 [==============================] - 80s 317ms/step - loss: 2.2637e-05 - accuracy: 1.0000 - val_loss: 0.1039 - val_accuracy: 0.9844
Epoch 23/60
252/252 [==============================] - 80s 316ms/step - loss: 1.7611e-05 - accuracy: 1.0000 - val_

In [ ]:
DetectionModel = '/home/richard/Desktop/BtainTumorDetectionSystem/Models/TumorDetectionModel'
Model.save(os.path.join(DetectionModel,'TumorDetectionModel.h5'))


In [ ]:
Model_Hist.history

In [ ]:
fig = plt.figure()
plt.plot(Model_Hist.history['loss'],color = 'red',label = 'Loss')
plt.plot(Model_Hist.history['val_loss'],color = 'blue', label = 'val loss')
plt.plot(Model_Hist.history['accuracy'], color = 'green', label = 'accuracy')
plt.legend(loc = 'upper right')
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(Model_Hist.history['accuracy'],color = 'green',label = 'Accuracy')
plt.plot(Model_Hist.history['val_accuracy'], color = 'blue', label = 'Vali Acc')
plt.legend(loc = 'upper right')
plt.show()


In [ ]:
import cv2
import tensorflow as tf

TestData = '/home/richard/Nts/BrainTumor/Brain Tumor Data Set/Brain Tumor/Cancer (132).jpg'
image_input = cv2.imread(TestData)
resizedimg = tf.image.resize(image_input,(256,256))
resizedimg.shape
img = plt.imshow(resizedimg.numpy().astype(int))
plt.show()

In [ ]:
predictions = Model.predict(np.expand_dims(resizedimg,0))
predictions

In [ ]:
from tensorflow.keras.models import load_model
import os
DetectionModel = '/home/richard/Desktop/BtainTumorDetectionSystem/Models/TumorDetectionModel'
New_Model = load_model(os.path.join(DetectionModel,'TumorDetectionModel.h5'))
pred = New_Model.predict(np.expand_dims(resizedimg,0))

if pred<0.5:
    print('Tumor detected!!!' )
    
elif pred >=0.5 and pred <=0.7:
    print('Redo the Test!!!')
elif pred >0.7 and pred<=1:
    print('No Tumor Detected!!!')

# Establishing tumor type.

In [18]:
GlioMeni = '/home/richard/Desktop/BtainTumorDetectionSystem/TumorTypes/GlioMeni'
GlioPitu = '/home/richard/Desktop/BtainTumorDetectionSystem/TumorTypes/GlioPitu'
MeniPitu = '/home/richard/Desktop/BtainTumorDetectionSystem/TumorTypes/MeniPitu'
GlioNeu = '/home/richard/Desktop/BtainTumorDetectionSystem/TumorTypes/GlioNeu'
MeniNeu = '/home/richard/Desktop/BtainTumorDetectionSystem/TumorTypes/MeniNeu'
NeuPitu = '/home/richard/Desktop/BtainTumorDetectionSystem/TumorTypes/NeuPitu'

GlioSchwa = '/home/richard/Desktop/BtainTumorDetectionSystem/TumorTypes/GlioSchwa'
MeniSchwa = '/home/richard/Desktop/BtainTumorDetectionSystem/TumorTypes/MeniSchwa'
PituSchwa = '/home/richard/Desktop/BtainTumorDetectionSystem/TumorTypes/PituSchwa'
NeuroSchwa = '/home/richard/Desktop/BtainTumorDetectionSystem/TumorTypes/NeuroSchwa'

In [19]:
GlMe = tf.keras.utils.image_dataset_from_directory(GlioMeni)

GlPi = tf.keras.utils.image_dataset_from_directory(GlioPitu)

MePi = tf.keras.utils.image_dataset_from_directory(MeniPitu)

GlNe = tf.keras.utils.image_dataset_from_directory(GlioNeu)

MeNe = tf.keras.utils.image_dataset_from_directory(MeniNeu)

NePi = tf.keras.utils.image_dataset_from_directory(NeuPitu)

GlSch = tf.keras.utils.image_dataset_from_directory(GlioSchwa)

MeSch = tf.keras.utils.image_dataset_from_directory(MeniSchwa)

PiSch = tf.keras.utils.image_dataset_from_directory(PituSchwa)

NeSch = tf.keras.utils.image_dataset_from_directory(NeuroSchwa)

Found 2982 files belonging to 2 classes.
Found 3229 files belonging to 2 classes.
Found 2484 files belonging to 2 classes.
Found 2050 files belonging to 2 classes.
Found 1455 files belonging to 2 classes.
Found 1719 files belonging to 2 classes.
Found 1984 files belonging to 2 classes.
Found 1221 files belonging to 2 classes.
Found 1651 files belonging to 2 classes.
Found 456 files belonging to 2 classes.


# Splitting datasets for predicting type of tumor evident

In [20]:
#splitting gliomas and Meningioma datasets
gmtra = int(len(GlMe)*.7)
gmva = int(len(GlMe)*.2)
gmte = int(len(GlMe)*.1)

gmtrain = GlMe.take(gmtra)
gmval = GlMe.skip(gmtra).take(gmva)
gmtest = GlMe.skip(gmtra + gmva).take(gmte)

#Splitting gliomas and Pituitary datasets
gptra = int(len(GlPi)*.7)
gpva = int(len(GlPi)*.2)
gpte = int(len(GlPi)*.1)

gptrain = GlPi.take(gptra)
gpval = GlPi.skip(gptra).take(gpva)
gptest = GlPi.skip(gptra + gpva).take(gpte)

#Splitting Meningioma and Pituitary datasets
mptra = int(len(MePi)*.7)
mpva = int(len(MePi)*.2)
mpte = int(len(MePi)*.1)

mptrain = MePi.take(mptra)
mpval = MePi.skip(mptra).take(mpva)
mptest = MePi.skip(mptra + mpva).take(mpte)

#Split Gliomas Neurocitoma

gntra = int(len(GlNe)*.7)
gnva = int(len(GlNe)*.2)
gnte = int(len(GlNe)*.1)

gntrain = GlNe.take(gntra)
gnval = GlNe.skip(gntra).take(gnva)
gntest = GlNe.skip(gntra + gnva).take(gnte)

#Split meningioma Neurocitoma

mntra = int(len(MeNe)*.7)
mnva = int(len(MeNe)*.2)
mnte = int(len(MeNe)*.1)


mntrain = MeNe.take(mntra)
mnval = MeNe.skip(mntra).take(mnva)
mntest = MeNe.skip(mntra + mnva).take(mnte)

#split Neurocitoma pituitary

nptra = int(len(NePi)*.7)
npva = int(len(NePi)*.2)
npte = int(len(NePi)*.1)

nptrain = NePi.take(nptra)
npval = NePi.skip(nptra).take(npva)
nptest = NePi.skip(nptra + npva).take(npte)

#Split Glioma Schwannoma
gstra = int(len(GlSch)*.7)
gsva = int(len(GlSch)*.2)
gste = int(len(GlSch)*.1)

gstrain = GlSch.take(gstra)
gsval = GlSch.skip(gstra).take(gsva)
gstest = GlSch.skip(gstra + gsva).take(gste)


#split Meningioma Schwannoma
mstra = int(len(MeSch)*.7)
msva = int(len(MeSch)*.2)
mste = int(len(MeSch)*.1)

mstrain = MeSch.take(mstra)
msval = MeSch.skip(mstra).take(msva)
mstest = MeSch.skip(mstra + msva).take(mste)


#Pituitary vs Schwannoma
pstra = int(len(PiSch)*.7)
psva = int(len(PiSch)*.2)
pste = int(len(PiSch)*.1)

pstrain = PiSch.take(pstra)
psval = PiSch.skip(pstra).take(psva)
pstest = PiSch.skip(pstra + psva).take(pste)

#Split Neurocitoma Schwannoma
nstra = int(len(NeSch)*.7)
nsva = int(len(NeSch)*.2)
nste = int(len(NeSch)*.1)

nstrain = NeSch.take(nstra)
nsval = NeSch.skip(nstra).take(nsva)
nstest = NeSch.skip(nstra + nsva).take(nste)







# Test for either glioma or meningioma

In [ ]:
#Creating a Sequential model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense,Flatten

GMModel = Sequential()

GMModel.add(Conv2D(32,(3,3),1,activation = 'relu',input_shape = (256,256,3)))
GMModel.add(MaxPooling2D())

GMModel.add(Conv2D(16,(3,3),1,activation = 'relu'))
GMModel.add(MaxPooling2D())

GMModel.add(Conv2D(32,(3,3),1,activation = 'relu'))
GMModel.add(MaxPooling2D())

GMModel.add(Flatten())

GMModel.add(Dense(256,activation = 'relu'))
GMModel.add(Dense(1,activation = 'sigmoid'))
GMModel.compile('adam',loss = tf.losses.BinaryCrossentropy(),metrics = ['accuracy'])

In [ ]:
GMModel.summary()

In [ ]:
GMlog = '/home/richard/Desktop/BtainTumorDetectionSystem/TypeLogFolder/GlMeLogs'

In [ ]:
T_CallBack = tf.keras.callbacks.TensorBoard(log_dir = GMlog)
GM_Hist = GMModel.fit(gmtrain,epochs = 20,validation_data = gmval,callbacks = [T_CallBack])

In [ ]:
#Saving Model
GliMe_Model = '/home/richard/Desktop/BtainTumorDetectionSystem/Models/GlioMeniModel'
GMModel.save(os.path.join(GliMe_Model,'gliomameningiomamodel.h5'))

# Test for either glioma or pituitary

In [ ]:
#Creating a Sequential model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense,Flatten

GPModel = Sequential()

GPModel.add(Conv2D(32,(3,3),1,activation = 'relu',input_shape = (256,256,3)))
GPModel.add(MaxPooling2D())

GPModel.add(Conv2D(16,(3,3),1,activation = 'relu'))
GPModel.add(MaxPooling2D())

GPModel.add(Conv2D(32,(3,3),1,activation = 'relu'))
GPModel.add(MaxPooling2D())

GPModel.add(Flatten())

GPModel.add(Dense(256,activation = 'relu'))
GPModel.add(Dense(1,activation = 'sigmoid'))
GPModel.compile('adam',loss = tf.losses.BinaryCrossentropy(),metrics = ['accuracy'])

In [ ]:
GPModel.summary()

In [ ]:
GPlog = '/home/richard/Desktop/BtainTumorDetectionSystem/TypeLogFolder/GlPilogs'

In [ ]:
GP_CallBack = tf.keras.callbacks.TensorBoard(log_dir = GPlog)
GP_Hist = GPModel.fit(gptrain,epochs = 30,validation_data = gpval,callbacks = [GP_CallBack])

In [ ]:
#Saving the model
GliPi_Model = '/home/richard/Desktop/BtainTumorDetectionSystem/Models/GlioPituModel'
GPModel.save(os.path.join(GliPi_Model,'gliomapituitarymodel.h5'))


# Test for either meningioma or pituitary

In [34]:
#Creating a Sequential model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense,Flatten

MPModel = Sequential()

MPModel.add(Conv2D(32,(3,3),1,activation = 'relu',input_shape = (256,256,3)))
MPModel.add(MaxPooling2D())

MPModel.add(Conv2D(16,(3,3),1,activation = 'relu'))
MPModel.add(MaxPooling2D())

MPModel.add(Conv2D(32,(3,3),1,activation = 'relu'))
MPModel.add(MaxPooling2D())

MPModel.add(Flatten())

MPModel.add(Dense(256,activation = 'relu'))
MPModel.add(Dense(1,activation = 'sigmoid'))
MPModel.compile('adam',loss = tf.losses.BinaryCrossentropy(),metrics = ['accuracy'])

In [35]:
MPModel.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 125, 125, 16)      4624      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 62, 62, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 60, 60, 32)        4640      
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 30, 30, 32)      

In [36]:
MPlog = '/home/richard/Desktop/BtainTumorDetectionSystem/TypeLogFolder/MePiLogs'

In [37]:
MP_Callback = tf.keras.callbacks.TensorBoard(log_dir = MPlog)
MPModel.fit(mptrain, epochs = 20, validation_data = mpval,callbacks = MP_Callback)

Epoch 1/50
54/54 [==============================] - 19s 308ms/step - loss: 15.9888 - accuracy: 0.7894 - val_loss: 0.2156 - val_accuracy: 0.9146
Epoch 2/50
54/54 [==============================] - 16s 294ms/step - loss: 0.1431 - accuracy: 0.9502 - val_loss: 0.1170 - val_accuracy: 0.9708
Epoch 3/50
54/54 [==============================] - 17s 315ms/step - loss: 0.0585 - accuracy: 0.9821 - val_loss: 0.1234 - val_accuracy: 0.9646
Epoch 4/50
54/54 [==============================] - 17s 313ms/step - loss: 0.0290 - accuracy: 0.9925 - val_loss: 0.0742 - val_accuracy: 0.9750
Epoch 5/50
54/54 [==============================] - 18s 331ms/step - loss: 0.0124 - accuracy: 0.9983 - val_loss: 0.0803 - val_accuracy: 0.9896
Epoch 6/50
54/54 [==============================] - 18s 334ms/step - loss: 0.0117 - accuracy: 0.9983 - val_loss: 0.0747 - val_accuracy: 0.9854
Epoch 7/50
54/54 [==============================] - 16s 298ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0792 - val_accuracy: 0.987

In [38]:
#Saving the model
MePi_Model = '/home/richard/Desktop/BtainTumorDetectionSystem/Models/meniPituModel'
MPModel.save(os.path.join(MePi_Model,'meningiomapituitarymodel.h5'))

# Glioma or Neurocitoma


In [ ]:
#Creating a Sequential model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense,Flatten

GNModel = Sequential()

GNModel.add(Conv2D(32,(3,3),1,activation = 'relu',input_shape = (256,256,3)))
GNModel.add(MaxPooling2D())

GNModel.add(Conv2D(16,(3,3),1,activation = 'relu'))
GNModel.add(MaxPooling2D())

GNModel.add(Conv2D(32,(3,3),1,activation = 'relu'))
GNModel.add(MaxPooling2D())

GNModel.add(Flatten())

GNModel.add(Dense(256,activation = 'relu'))
GNModel.add(Dense(1,activation = 'sigmoid'))
GNModel.compile('adam',loss = tf.losses.BinaryCrossentropy(),metrics = ['accuracy'])

In [ ]:
GNModel.summary()

In [ ]:
GNLog = '/home/richard/Desktop/BtainTumorDetectionSystem/TypeLogFolder/GlNelogs'

GN_callback = tf.keras.callbacks.TensorBoard(log_dir = 'GNLog') 

GNModel.fit(gntrain, epochs = 20, validation_data = gnval, callbacks = GN_callback)

In [ ]:
GlioNeuMod = '/home/richard/Desktop/BtainTumorDetectionSystem/Models/GlioNeuModel'
GNModel.save(os.path.join(GlioNeuMod,'gliomaneurocitomamodel.h5'))

# Meningioma or Neurocitoma


In [43]:
#Creating a Sequential model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense,Flatten

MNModel = Sequential()

MNModel.add(Conv2D(32,(3,3),1,activation = 'relu',input_shape = (256,256,3)))
MNModel.add(MaxPooling2D())

MNModel.add(Conv2D(16,(3,3),1,activation = 'relu'))
MNModel.add(MaxPooling2D())

MNModel.add(Conv2D(32,(3,3),1,activation = 'relu'))
MNModel.add(MaxPooling2D())

MNModel.add(Flatten())

MNModel.add(Dense(256,activation = 'relu'))
MNModel.add(Dense(1,activation = 'sigmoid'))
MNModel.compile('adam',loss = tf.losses.BinaryCrossentropy(),metrics = ['accuracy'])

In [44]:
MNModel.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 127, 127, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 125, 125, 16)      4624      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 62, 62, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 60, 60, 32)        4640      
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 30, 30, 32)      

In [45]:
MNLog = "/home/richard/Desktop/BtainTumorDetectionSystem/TypeLogFolder/MeNeLog"

MN_Callback = tf.keras.callbacks.TensorBoard(log_dir = MNLog)

MNModel.fit(mntrain, epochs = 20, validation_data = mnval, callbacks = MN_Callback)

Epoch 1/50
32/32 [==============================] - 12s 303ms/step - loss: 17.1701 - accuracy: 0.7959 - val_loss: 0.3198 - val_accuracy: 0.9201
Epoch 2/50
32/32 [==============================] - 10s 304ms/step - loss: 0.1682 - accuracy: 0.9531 - val_loss: 0.1402 - val_accuracy: 0.9653
Epoch 3/50
32/32 [==============================] - 10s 324ms/step - loss: 0.0751 - accuracy: 0.9736 - val_loss: 0.0437 - val_accuracy: 0.9792
Epoch 4/50
32/32 [==============================] - 10s 310ms/step - loss: 0.0283 - accuracy: 0.9922 - val_loss: 0.0223 - val_accuracy: 0.9931
Epoch 5/50
32/32 [==============================] - 10s 313ms/step - loss: 0.0152 - accuracy: 0.9971 - val_loss: 0.0119 - val_accuracy: 1.0000
Epoch 6/50
32/32 [==============================] - 9s 294ms/step - loss: 0.0055 - accuracy: 1.0000 - val_loss: 0.0067 - val_accuracy: 0.9965
Epoch 7/50
32/32 [==============================] - 10s 313ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0057 - val_accuracy: 0.9965

In [46]:
MeniNeuMod = "/home/richard/Desktop/BtainTumorDetectionSystem/Models/MeniNeuModel"
MNModel.save(os.path.join(MeniNeuMod,'meningiomaneurocitomamodel.h5'))

# Neurocitoma or pituitary

In [47]:
#Creating a Sequential model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense,Flatten

NPModel = Sequential()

NPModel.add(Conv2D(32,(3,3),1,activation = 'relu',input_shape = (256,256,3)))
NPModel.add(MaxPooling2D())

NPModel.add(Conv2D(16,(3,3),1,activation = 'relu'))
NPModel.add(MaxPooling2D())

NPModel.add(Conv2D(32,(3,3),1,activation = 'relu'))
NPModel.add(MaxPooling2D())

NPModel.add(Flatten())

NPModel.add(Dense(256,activation = 'relu'))
NPModel.add(Dense(1,activation = 'sigmoid'))
NPModel.compile('adam',loss = tf.losses.BinaryCrossentropy(),metrics = ['accuracy'])

In [48]:
NPModel.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 127, 127, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 125, 125, 16)      4624      
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 62, 62, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 60, 60, 32)        4640      
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 30, 30, 32)      

In [49]:
NPLog = "/home/richard/Desktop/BtainTumorDetectionSystem/TypeLogFolder/NePiLogs"

NP_Callback = tf.keras.callbacks.TensorBoard(log_dir = NPLog)

NPModel.fit(nptrain, epochs = 20, validation_data = npval, callbacks = NP_Callback)

Epoch 1/50
37/37 [==============================] - 13s 308ms/step - loss: 7.4092 - accuracy: 0.8910 - val_loss: 0.1379 - val_accuracy: 0.9906
Epoch 2/50
37/37 [==============================] - 11s 310ms/step - loss: 0.0325 - accuracy: 0.9958 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 3/50
37/37 [==============================] - 11s 292ms/step - loss: 0.0100 - accuracy: 0.9983 - val_loss: 0.0036 - val_accuracy: 0.9969
Epoch 4/50
37/37 [==============================] - 11s 305ms/step - loss: 0.0016 - accuracy: 0.9992 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 5/50
37/37 [==============================] - 11s 297ms/step - loss: 0.0019 - accuracy: 0.9992 - val_loss: 3.0420e-04 - val_accuracy: 1.0000
Epoch 6/50
37/37 [==============================] - 11s 295ms/step - loss: 1.3928e-04 - accuracy: 1.0000 - val_loss: 1.2926e-04 - val_accuracy: 1.0000
Epoch 7/50
37/37 [==============================] - 11s 305ms/step - loss: 8.1483e-05 - accuracy: 1.0000 - val_loss: 7.5499e-05 - 

In [50]:
NeuPituMod = "/home/richard/Desktop/BtainTumorDetectionSystem/Models/NeuPituModel"

NPModel.save(os.path.join(NeuPituMod, 'neurocitomapituitarymodel.h5'))

# Glioma  Vs Schwannoma

In [ ]:
#Creating a Sequential model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense,Flatten

GSModel = Sequential()

GSModel.add(Conv2D(32,(3,3),1,activation = 'relu',input_shape = (256,256,3)))
GSModel.add(MaxPooling2D())

GSModel.add(Conv2D(16,(3,3),1,activation = 'relu'))
GSModel.add(MaxPooling2D())

GSModel.add(Conv2D(32,(3,3),1,activation = 'relu'))
GSModel.add(MaxPooling2D())

GSModel.add(Flatten())

GSModel.add(Dense(256,activation = 'relu'))
GSModel.add(Dense(1,activation = 'sigmoid'))
GSModel.compile('adam',loss = tf.losses.BinaryCrossentropy(),metrics = ['accuracy'])

In [ ]:
GSModel.summary()

In [ ]:
GSLog = "/home/richard/Desktop/BtainTumorDetectionSystem/TypeLogFolder/GlSchwLogs"

GS_Callback = tf.keras.callbacks.TensorBoard(log_dir = GSLog)

GSModel.fit(gstrain, epochs = 20, validation_data = gsval, callbacks = GS_Callback)

In [ ]:
GSMod = "/home/richard/Desktop/BtainTumorDetectionSystem/Models/GlioSchwaModel"
GSModel.save(os.path.join(GSMod,'gliomascwannomamodel.h5'))

# Meningioma Vs Schwannoma



In [ ]:
#Creating a Sequential model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense,Flatten

MSModel = Sequential()

MSModel.add(Conv2D(32,(3,3),1,activation = 'relu',input_shape = (256,256,3)))
MSModel.add(MaxPooling2D())

MSModel.add(Conv2D(16,(3,3),1,activation = 'relu'))
MSModel.add(MaxPooling2D())

MSModel.add(Conv2D(32,(3,3),1,activation = 'relu'))
MSModel.add(MaxPooling2D())

MSModel.add(Flatten())

MSModel.add(Dense(256,activation = 'relu'))
MSModel.add(Dense(1,activation = 'sigmoid'))
MSModel.compile('adam',loss = tf.losses.BinaryCrossentropy(),metrics = ['accuracy'])

In [ ]:
MSModel.summary()

In [ ]:
MSLog = '/home/richard/Desktop/BtainTumorDetectionSystem/TypeLogFolder/MeSchwaLogs'
MS_Callback = tf.keras.callbacks.TensorBoard(log_dir = MSLog)

MSModel.fit(mstrain, epochs = 20, validation_data = msval,callbacks = MS_Callback)

In [ ]:
MSMod = '/home/richard/Desktop/BtainTumorDetectionSystem/Models/MeniSchwaModel'
MSModel.save(os.path.join(MSMod,'meningiomaschwannomamodel.h5'))

# Pituitary Vs Schwannoma

In [28]:
#Creating a Sequential model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense,Flatten

PSModel = Sequential()

PSModel.add(Conv2D(32,(3,3),1,activation = 'relu',input_shape = (256,256,3)))
PSModel.add(MaxPooling2D())

PSModel.add(Conv2D(16,(3,3),1,activation = 'relu'))
PSModel.add(MaxPooling2D())

PSModel.add(Conv2D(32,(3,3),1,activation = 'relu'))
PSModel.add(MaxPooling2D())

PSModel.add(Flatten())

PSModel.add(Dense(1,activation = 'sigmoid'))
PSModel.compile('adam',loss = tf.losses.BinaryCrossentropy(),metrics = ['accuracy'])

In [29]:
PSModel.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 125, 125, 16)      4624      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 62, 62, 16)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 60, 60, 32)        4640      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 30, 30, 32)      

In [30]:
PSLog = '/home/richard/Desktop/BtainTumorDetectionSystem/TypeLogFolder/PiSchwaLogs'
PS_Callback = tf.keras.callbacks.TensorBoard(log_dir = PSLog)
PSModel.fit(pstrain,epochs = 20, validation_data = psval,callbacks = PS_Callback)

Epoch 1/20
36/36 [==============================] - 12s 262ms/step - loss: 3.2937 - accuracy: 0.8993 - val_loss: 0.2631 - val_accuracy: 0.9344
Epoch 2/20
36/36 [==============================] - 9s 252ms/step - loss: 0.0778 - accuracy: 0.9818 - val_loss: 0.0152 - val_accuracy: 0.9937
Epoch 3/20
36/36 [==============================] - 9s 251ms/step - loss: 0.0255 - accuracy: 0.9913 - val_loss: 0.0378 - val_accuracy: 0.9844
Epoch 4/20
36/36 [==============================] - 9s 250ms/step - loss: 0.0063 - accuracy: 0.9983 - val_loss: 0.0027 - val_accuracy: 1.0000
Epoch 5/20
36/36 [==============================] - 9s 245ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0031 - val_accuracy: 1.0000
Epoch 6/20
36/36 [==============================] - 9s 243ms/step - loss: 9.6085e-04 - accuracy: 1.0000 - val_loss: 0.0041 - val_accuracy: 0.9969
Epoch 7/20
36/36 [==============================] - 9s 243ms/step - loss: 6.3295e-04 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.00

In [31]:
PSMod = '/home/richard/Desktop/BtainTumorDetectionSystem/Models/PituSchwaModel'

PSModel.save(os.path.join(PSMod,'pituitaryschwannomamodel.h5'))

# Neurocitoma Vs Schwannoma

In [21]:
#Creating a Sequential model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense,Flatten

NSModel = Sequential()

NSModel.add(Conv2D(32,(3,3),1,activation = 'relu',input_shape = (256,256,3)))
NSModel.add(MaxPooling2D())

NSModel.add(Conv2D(16,(3,3),1,activation = 'relu'))
NSModel.add(MaxPooling2D())

NSModel.add(Conv2D(32,(3,3),1,activation = 'relu'))
NSModel.add(MaxPooling2D())

NSModel.add(Flatten())

NSModel.add(Dense(256,activation = 'relu'))
NSModel.add(Dense(1,activation = 'sigmoid'))
NSModel.compile('adam',loss = tf.losses.BinaryCrossentropy(),metrics = ['accuracy'])

In [22]:
NSModel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 125, 125, 16)      4624      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 62, 62, 16)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 60, 60, 32)        4640      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 30, 30, 32)      

In [23]:
NSLog = '/home/richard/Desktop/BtainTumorDetectionSystem/TypeLogFolder/NeuSchwaLogs'
NS_Callback = tf.keras.callbacks.TensorBoard(log_dir = NSLog)
NSModel.fit(nstrain,epochs = 20, validation_data = nsval,callbacks = NS_Callback)

Epoch 1/20
10/10 [==============================] - 6s 324ms/step - loss: 91.1785 - accuracy: 0.5938 - val_loss: 6.9284 - val_accuracy: 0.6875
Epoch 2/20
10/10 [==============================] - 3s 295ms/step - loss: 2.8084 - accuracy: 0.8062 - val_loss: 0.1726 - val_accuracy: 0.9792
Epoch 3/20
10/10 [==============================] - 3s 295ms/step - loss: 0.3761 - accuracy: 0.9500 - val_loss: 0.0677 - val_accuracy: 0.9896
Epoch 4/20
10/10 [==============================] - 3s 297ms/step - loss: 0.0781 - accuracy: 0.9844 - val_loss: 0.2064 - val_accuracy: 0.9688
Epoch 5/20
10/10 [==============================] - 3s 293ms/step - loss: 0.0207 - accuracy: 0.9875 - val_loss: 0.0611 - val_accuracy: 0.9792
Epoch 6/20
10/10 [==============================] - 3s 295ms/step - loss: 0.0062 - accuracy: 0.9969 - val_loss: 7.0682e-05 - val_accuracy: 1.0000
Epoch 7/20
10/10 [==============================] - 3s 298ms/step - loss: 1.4356e-04 - accuracy: 1.0000 - val_loss: 0.0573 - val_accuracy: 0.98

In [24]:
NSMod = '/home/richard/Desktop/BtainTumorDetectionSystem/Models/NeuSchwaModel'
NSModel.save(os.path.join(NSMod,'neurocitomaschwannomamodel.h5'))

# Model Accuracy

In [25]:
Det = '/home/richard/Desktop/BtainTumorDetectionSystem/Models/TumorDetectionModel'

Tumor = load_model(os.path.join(Det,'TumorDetectionModel.h5'))

In [26]:
accuracy = Tumor.evaluate(Test)

42/42 [==============================] - 151s 289ms/step - loss: 0.1240 - accuracy: 0.9939


In [27]:
print(accuracy)

[0.12398116290569305, 0.9939393997192383]
